In [1]:
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

#### This notebooks contains data pre-processing of merged traffic density columns using Pandas. 
##### * First, we will remove data for all GEOHASH that has less than 28k hours of data.
##### * Then, we will transform the dataset to ensure every remaining GEOHASH location has 29,184 hours of data points. 

##### At present, for each GEOHASH, the DATE_TIME column is not in propert hour frequency. An exmaple will be shown soon.


In [2]:
%%time

# Read Parquet file and choose only selected columns (Ignore date time features for now to save memory)
selected_cols=['DATE_TIME', 'LATITUDE', 'LONGITUDE', 'GEOHASH', 'MINIMUM_SPEED', 'MAXIMUM_SPEED', 'AVERAGE_SPEED', 'NUMBER_OF_VEHICLES']

raw_df = (
    pd.read_parquet("datasets/01_tr_density/ist_traffic_density_rev01.gz", columns=selected_cols)
)

Wall time: 19.5 s


In [3]:
%%time

# Creating a copy to save time in case the original dataset is needed and to avoid using pd.read_parquet due to the time taken to read the file.
all_df=raw_df.copy()

Wall time: 1.56 s


In [4]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66146858 entries, 0 to 66146857
Data columns (total 8 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DATE_TIME           datetime64[ns]
 1   LATITUDE            float32       
 2   LONGITUDE           float32       
 3   GEOHASH             object        
 4   MINIMUM_SPEED       uint8         
 5   MAXIMUM_SPEED       uint8         
 6   AVERAGE_SPEED       uint8         
 7   NUMBER_OF_VEHICLES  uint16        
dtypes: datetime64[ns](1), float32(2), object(1), uint16(1), uint8(3)
memory usage: 1.8+ GB


In [5]:
# Showing a sample of the dataset
all_df.query("GEOHASH=='sxk9w1'").set_index('DATE_TIME').head()

,LATITUDE,LONGITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
DATE_TIME,,,,,,,
2020-01-01 00:00:00,41.053162,29.09729,sxk9w1,46,7,24,7
2020-01-01 01:00:00,41.053162,29.09729,sxk9w1,25,17,21,2
2020-01-01 02:00:00,41.053162,29.09729,sxk9w1,44,15,29,5
2020-01-01 04:00:00,41.053162,29.09729,sxk9w1,61,9,33,5
2020-01-01 05:00:00,41.053162,29.09729,sxk9w1,42,8,26,2


#### The above sample of the datasets shows the first 5 rows of GEOHASH='sxk9w1'. Looking at the DATE_TIME column, we can see there is no row for 03:00 hours. Similarly for other GEOHASH, there are multiple missing hours of data. Sometimes there are even missing data for whole days. 

#### Later, we will ensure all GEOHASH has 29,184 rows.

In [6]:
%%time
# Creating a seperate dataframe that shows all unique GEOHASH and the cooresponding count of rows. 
# Followed by creating percentage of rows available compared to total number of hours from Jan 2020 to April 2023 (29,184 hours)

full_hours = 29184

geohash_hrs = all_df.groupby("GEOHASH", as_index=False).size()
geohash_hrs['perc'] = np.round(100*(geohash_hrs['size']/full_hours), 3)

Wall time: 7.3 s


In [7]:
%%time
# Checking GEOHASH with atleast 96% non null rows. The corresponsing number of available data is displayed in the `size` column
geohash_hrs[geohash_hrs['perc'].ge(96)].sort_values('perc')

Wall time: 0 ns


,GEOHASH,size,perc
3318,sxk9w1,28017,96.001
1025,sxk3he,28017,96.001
4473,sxkd8d,28018,96.005
3235,sxk9tb,28025,96.029
3239,sxk9tf,28026,96.032
...,...,...,...
1198,sxk3rs,28514,97.704
1342,sxk3xe,28514,97.704
1183,sxk3r9,28515,97.708
1119,sxk3nk,28515,97.708


In [8]:
%%time
# Saving those GEOHASH with 96% hours of data
selected_gh = geohash_hrs.loc[geohash_hrs['perc'].ge(96), 'GEOHASH'].values

Wall time: 0 ns


In [9]:
%%time
# Filtering out GEOHASH
filtered_df = (
    all_df.loc[all_df['GEOHASH'].isin(selected_gh)] # select only GEOHASH present in selected_gh variable
    .sort_values(['DATE_TIME', 'GEOHASH'])
)

Wall time: 30.9 s


In [10]:
# Number of rows reduced from 66+ million records to 51.5 million records
# The number of rows in the filtered dataset will increase again laterwards when ensuring the DATE_TIME for all GEOHASH is of hourly frequency
len(filtered_df)

51548379

In [11]:
# Checking whether all GEOHASH has start DATE_TIME as "2020-01-01 00:00:00"
(
    filtered_df.groupby('GEOHASH')['DATE_TIME'].first()
    .sort_values(ascending=False)
    .head()
)

GEOHASH
sx7ckc   2020-01-01 02:00:00
sx7cm5   2020-01-01 02:00:00
sx7chk   2020-01-01 00:00:00
sxk9pj   2020-01-01 00:00:00
sxk9pz   2020-01-01 00:00:00
Name: DATE_TIME, dtype: datetime64[ns]

#### the above dataframe shows that 2 GEOHASH has start datetime at 02:00 hours instead of 00:00 hours.

In [12]:
filtered_df.loc[filtered_df['GEOHASH'].isin(["sx7ckc", "sx7cm5"]), ['GEOHASH', 'LATITUDE', 'LONGITUDE']].drop_duplicates()

,GEOHASH,LATITUDE,LONGITUDE
4611,sx7ckc,41.009216,27.987671
5319,sx7cm5,41.020203,27.998657


In [13]:
filtered_df.head()

,DATE_TIME,LATITUDE,LONGITUDE,GEOHASH,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
1367,2020-01-01,40.981750,27.965698,sx7chk,112,86,99,2
1750,2020-01-01,40.987244,27.965698,sx7chm,124,63,92,3
471,2020-01-01,40.987244,27.976685,sx7cht,103,17,42,3
1406,2020-01-01,40.992737,27.976685,sx7chw,117,26,76,6
1645,2020-01-01,40.998230,27.976685,sx7chx,139,23,81,5


##### Creating a separate dataframe with the required rows and append to the filtered_df.

In [14]:
extra_rows = []

for gh, lat, lon in zip(["sx7ckc", "sx7cm5"], [41.009216, 41.020203], [27.987671, 27.998657]):
    for dt in ["2020-01-01 00:00:00", "2020-01-01 01:00:00"]:
        extra_rows.append({
            "DATE_TIME" : pd.to_datetime(dt),
            "LATITUDE" : lat,
            "LONGITUDE" : lon,
            "GEOHASH" : gh
        })
    
extra_rows_df = pd.DataFrame(extra_rows)
extra_rows_df

,DATE_TIME,LATITUDE,LONGITUDE,GEOHASH
0,2020-01-01 00:00:00,41.009216,27.987671,sx7ckc
1,2020-01-01 01:00:00,41.009216,27.987671,sx7ckc
2,2020-01-01 00:00:00,41.020203,27.998657,sx7cm5
3,2020-01-01 01:00:00,41.020203,27.998657,sx7cm5


In [15]:
filtered_df2 = pd.concat([filtered_df, extra_rows_df], ignore_index=True)

In [16]:
%%time
# Here we will transform the dataset to ensure hourly frequency across all GEOHASH. 
# There will be null values for some of the rows as a result. These will be filled in using appropriate methods later.

df = (
    filtered_df2.set_index('DATE_TIME') # set DATE_TIME as inde so that resampling can be done
    .groupby('GEOHASH') 
    .resample('H').sum(min_count=1) # ensuring all rows for each GEOHASH is oh hour 1 hour. Ensure to keep null values as it is (no filling null values)
    .drop('GEOHASH', axis=1) # an extra GEOHASH column in created as a result. Drop it.
    .reset_index() # To retain the GEOHASH and DATE_TIME columns
    .assign(LATITUDE=lambda x: x['LATITUDE'].ffill(), 
            LONGITUDE=lambda x: x['LONGITUDE'].ffill()) # Fill any missing rows in the coords column using forward fill.
)

Wall time: 2min 23s


In [17]:
df

,GEOHASH,DATE_TIME,LATITUDE,LONGITUDE,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
0,sx7chk,2020-01-01 00:00:00,40.981750,27.965698,112.0,86.0,99.0,2.0
1,sx7chk,2020-01-01 01:00:00,40.981750,27.965698,98.0,98.0,98.0,1.0
2,sx7chk,2020-01-01 02:00:00,40.981750,27.965698,141.0,60.0,92.0,3.0
3,sx7chk,2020-01-01 03:00:00,40.981750,27.965698,100.0,79.0,81.0,2.0
4,sx7chk,2020-01-01 04:00:00,40.981750,27.965698,80.0,64.0,69.0,2.0
...,...,...,...,...,...,...,...,...
52910587,sxkf45,2023-04-30 19:00:00,41.152039,29.273071,171.0,69.0,102.0,124.0
52910588,sxkf45,2023-04-30 20:00:00,41.152039,29.273071,153.0,64.0,94.0,109.0
52910589,sxkf45,2023-04-30 21:00:00,41.152039,29.273071,146.0,64.0,94.0,89.0
52910590,sxkf45,2023-04-30 22:00:00,41.152039,29.273071,146.0,64.0,90.0,96.0


In [18]:
%%time
# there are about 1.3 million rows of missing values in total
df.isnull().sum()

Wall time: 3.54 s


GEOHASH                     0
DATE_TIME                   0
LATITUDE                    0
LONGITUDE                   0
MINIMUM_SPEED         1362213
MAXIMUM_SPEED         1362213
AVERAGE_SPEED         1362213
NUMBER_OF_VEHICLES    1362213
dtype: int64

In [19]:
# The number of rows has increased from approximatly 51.5 million rows for the filtered dataset to 52.9 million rows for the transformed dataset
len(df)

52910592

In [20]:
df.query("GEOHASH=='sxk9w1'").head()

,GEOHASH,DATE_TIME,LATITUDE,LONGITUDE,MINIMUM_SPEED,MAXIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
39077376,sxk9w1,2020-01-01 00:00:00,41.053162,29.09729,46.0,7.0,24.0,7.0
39077377,sxk9w1,2020-01-01 01:00:00,41.053162,29.09729,25.0,17.0,21.0,2.0
39077378,sxk9w1,2020-01-01 02:00:00,41.053162,29.09729,44.0,15.0,29.0,5.0
39077379,sxk9w1,2020-01-01 03:00:00,41.053162,29.09729,NaN,NaN,NaN,NaN
39077380,sxk9w1,2020-01-01 04:00:00,41.053162,29.09729,61.0,9.0,33.0,5.0


#### As seen from the above sample of the dataset for GEOHASH='sxk9w1' (same as shown earlier), now there is a row of 03:00 hours. The corresponding values for speed and num_of_vehicles is null. We will handle missing values in the next section.

In [21]:
df.groupby("GEOHASH").size().sort_values()

GEOHASH
sx7chk    29184
sxk9px    29184
sxk9pw    29184
sxk9pt    29184
sxk9ps    29184
          ...  
sxk6ms    29184
sxk6mr    29184
sxk6mq    29184
sxk6px    29184
sxkf45    29184
Length: 1813, dtype: int64